# Profiling with Synthetic Vortex Example

This notebook is used for profiling the vortector with the synthetic vortex example.

In [1]:
import os
from subprocess import run
res = run(["git", "rev-parse", "--show-toplevel"], capture_output=True)
root_dir = res.stdout.decode().strip()
os.chdir(root_dir)

In [2]:
import numpy as np
import matplotlib.pyplot as plt

%reload_ext autoreload
%autoreload 2

from src.vortector.vortector import Vortector
from synthvortex import VortexGenerator

import cProfile
import pstats
from pstats import SortKey
from tempfile import NamedTemporaryFile

## Creating a synthetic vortex

Vortices in PPDs appear as 2D Gaussians in the vortensity (minimum) and the surface density (maximum), we'll call it density from now on.

For a synthetic vortex, all we need to do is to set up a grid and put in a Gaussian distribution in both the vortensity and surface density.

In [18]:
scaling = 1
vgen = VortexGenerator(3, periodic_y=True, Nr=121*scaling,Nphi=251*scaling,
                       seed=123009799731638641813521844717530781350)

In [19]:
vd = Vortector(vgen.Rs, vgen.Phis, vgen.Area, vgen.vortensity, vgen.density, verbose=True)
vd.detect()

Number of found contours: 103
Number of closed contours: 94
Area of contours: [14571.5, 14571.5, 12543.0, 12543.0, 11110.5, 11110.5, 10027.0, 10027.0, 9147.5, 9147.5, 4959.0, 4959.0, 4608.5, 4608.5, 4269.5, 4269.5, 3972.0, 3972.0, 3677.0, 3677.0, 3384.0, 3384.0, 3351.5, 3351.5, 3115.0, 3115.0, 3037.5, 3037.5, 2839.0, 2839.0, 2769.0, 2769.0, 2575.5, 2575.5, 2529.0, 2529.0, 2315.5, 2315.5, 2307.0, 2307.0, 2105.5, 2105.5, 2057.0, 2057.0, 1907.5, 1907.5, 1792.5, 1792.5, 1721.5, 1721.5, 1535.5, 1535.5, 1515.5, 1515.5, 1344.5, 1344.5, 1230.0, 1218.0, 1218.0, 1167.5, 1167.5, 1164.0, 978.5, 978.5, 921.5, 921.5, 792.0, 792.0, 607.0, 607.0, 591.5, 591.5, 390.0, 390.0, 272.0, 263.5, 263.5, 178.0, 178.0, 176.5, 176.5, 137.5, 137.5, 75.0, 75.0, 62.5, 62.5, 54.5, 54.5, 51.0, 38.0, 38.0, 34.5, 34.5]
Removed 45 contours which were duplicates. 49 remaining.
Detected 24 elliptic contours.
Removed 0 candidates by common area and bbox. 24 remaining
Removed 3 candidates by due to vortensity drop < 0.1.


[{'contour': {'detection': {'contour_value': 0.7000000000000002,
    'opencv_contour_number': 34000,
    'uuid': 'da5cb90c-f6cf-4b76-942c-ea9ae08dde0d',
    'pixel_area': 3351.5,
    'pixel_arcLength': 238.06601536273956,
    'bounding_x_img': array([ 52, 102], dtype=int32),
    'bounding_y_img': array([287, 378], dtype=int32),
    'pnt_xlow_img': array([ 52, 332], dtype=int32),
    'pnt_xhigh_img': array([102, 339], dtype=int32),
    'pnt_ylow_img': array([ 77, 287], dtype=int32),
    'pnt_yhigh_img': array([ 86, 378], dtype=int32),
    'dx_img': 50,
    'dy_img': 91,
    'pnt_xlow': (52, 207),
    'pnt_xhigh': (102, 214),
    'pnt_ylow': (77, 162),
    'pnt_yhigh': (86, 2),
    'bbox_inds': array([[ 52, 207],
           [102, 214],
           [ 77, 162],
           [ 86,   2]]),
    'ellipse': {'center_img': (334.06097412109375, 78.55438232421875),
     'axesLengths_img': (47.66494369506836, 90.572265625),
     'angle_img': 93.4156723022461},
    'ellipse_area_delta_relative': 0.0729

## Profile

We will use the snakeviz package.
If it's not installed, run 

In [26]:
#!python3 -m pip install snakeviz
%load_ext snakeviz
vd.verbose = False
%snakeviz vd.detect(autofit=True)

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz
 
*** Profile stats marshalled to file '/tmp/tmpwsojy9_z'. 
Embedding SnakeViz in this document...


## Profiling Vortex Generator

In [29]:
from synthvortex import VortexGenerator
%reload_ext autoreload
%autoreload 2
%load_ext snakeviz
scaling = 10
%snakeviz VortexGenerator(3, periodic_y=True, Nr=121*scaling,Nphi=251*scaling)

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz
 
*** Profile stats marshalled to file '/tmp/tmpyfmfkepf'. 
Embedding SnakeViz in this document...


## Profiling Vortensity Calculation

In [4]:
%load_ext snakeviz

In [14]:
from simdata_vorticity import provide_simulation_data
simid = "3625e016"
Noutput = 202
%snakeviz provide_simulation_data(simid, Noutput, skip_cache=True)

 
*** Profile stats marshalled to file '/tmp/tmps1zugak_'. 
Embedding SnakeViz in this document...
